# Initialisation

In [130]:
import os
import torch
os.environ['TORCH'] = torch.__version__
from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv
from torch_geometric.transforms import NormalizeFeatures

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

Dataframe_Labels = pd.read_csv("../../BLCA_DATA/Workspace/labels_str.csv")
Dataframe_link = pd.read_csv("../../BLCA_DATA/Workspace/patient_norm.csv")
Dataframe_node= pd.read_csv("../../BLCA_DATA/Workspace/node_embedding.csv")

Dataframe_Labels['class_int'], uniques = pd.factorize(Dataframe_Labels['class'])
Dataframe_Labels

,Patient,class,class_int
0,TCGA-2F-A9KO,LumP,0
1,TCGA-2F-A9KP,LumP,0
2,TCGA-2F-A9KQ,LumP,0
3,TCGA-2F-A9KR,Ba/Sq,1
4,TCGA-2F-A9KT,Ba/Sq,1
...,...,...,...
399,TCGA-ZF-AA56,Ba/Sq,1
400,TCGA-ZF-AA58,Ba/Sq,1
401,TCGA-ZF-AA5H,Ba/Sq,1
402,TCGA-ZF-AA5N,LumP,0


# Mise en place des arretes, noeuds et leur poids

### à améliorer, modification du treshold

In [131]:
node_features = Dataframe_node.drop(columns=['Patient']).values
node_features = torch.tensor(node_features, dtype=torch.float)

x = node_features
patient_similarity = cosine_similarity(Dataframe_link.iloc[:, 1:])
similarity_threshold = 0.5  # Exemple de seuil de similarité

edge_index = []
edge_attr = []

for i in range(patient_similarity.shape[0]):
    for j in range(i + 1, patient_similarity.shape[0]):
        if patient_similarity[i, j] > similarity_threshold:
            edge_index.append([i, j])
            edge_attr.append((patient_similarity[i, j] - similarity_threshold)/(1 - similarity_threshold))
        patient_similarity[i, i] = 0

edge_index = torch.tensor(edge_index, dtype=torch.int64).t().contiguous()
edge_features = torch.tensor(Dataframe_link.drop(columns=['Patient']).values, dtype=torch.float)
edge_attr = torch.tensor(edge_attr, dtype=torch.float)

temporary_node_tab = Dataframe_Labels["class_int"].values
node_labels = torch.tensor(temporary_node_tab, dtype=torch.long)
node_labels

tensor([0, 0, 0, 1, 1, 2, 2, 3, 0, 1, 2, 0, 1, 0, 2, 1, 1, 0, 1, 1, 2, 0, 2, 4,
        1, 1, 5, 1, 1, 1, 1, 1, 1, 0, 3, 3, 0, 1, 1, 2, 1, 5, 4, 1, 0, 1, 1, 0,
        1, 1, 5, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 2, 1, 2, 0, 0, 1, 2, 1, 2, 0, 0, 1, 2, 4, 1, 3, 0,
        3, 4, 1, 1, 1, 4, 2, 1, 1, 3, 0, 4, 1, 2, 1, 1, 3, 0, 1, 0, 0, 2, 4, 1,
        0, 1, 1, 0, 1, 1, 1, 3, 2, 5, 0, 0, 1, 2, 0, 0, 1, 1, 0, 1, 0, 2, 0, 0,
        0, 0, 0, 0, 0, 3, 2, 0, 0, 1, 0, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 2, 3, 3, 1, 1, 1, 3, 2, 1, 2, 2, 3, 3, 0, 1, 0, 3, 1, 1, 0, 3, 1, 3,
        3, 1, 1, 0, 1, 1, 4, 3, 3, 1, 0, 3, 3, 1, 4, 1, 2, 0, 2, 3, 0, 1, 0, 1,
        1, 0, 5, 0, 1, 1, 0, 2, 0, 1, 0, 2, 0, 1, 3, 0, 1, 0, 1, 1, 2, 1, 2, 4,
        4, 1, 1, 0, 2, 2, 1, 0, 1, 0, 1, 1, 0, 3, 2, 1, 4, 0, 2, 2, 0, 3, 2, 0,
        0, 1, 2, 0, 2, 0, 0, 2, 1, 1, 3, 4, 1, 1, 1, 0, 1, 1, 1, 1, 2, 3, 3, 0,
        0, 0, 2, 0, 2, 1, 2, 0, 2, 1, 1,

In [132]:
classes_dict = {0: 'LumP', 
                1: 'Ba/Sq', 
                2: 'LumU', 
                3: 'Stroma-rich', 
                4: 'LumNS', 
                5: 'NE-like'
}

def count_classes_weights(tensor):
    array = tensor.numpy()
    N = np.shape(array)[0]
    classes_tab = { 0: 0, 
                    1: 0,
                    2: 0, 
                    3: 0, 
                    4: 0,
                    5: 0
    }
    for i in array:
        classes_tab[i]+=1

    mean_nb_classes = 0
    for i in classes_tab:
        mean_nb_classes += i
    mean_nb_classes *= 1/len(classes_tab)
    
    # normalize the weights
    weight_sum = 0
    for i in range(len(classes_tab)):
        if classes_tab[i] != 0:
            weight_sum += mean_nb_classes / classes_tab[i]
    alpha = 1 / weight_sum

    weight_dict = {}
    used_classes = []
    for i in range(len(classes_tab)):
        if classes_tab[i] != 0:
            weight_dict[i] = alpha * (mean_nb_classes / classes_tab[i]) *50
            used_classes.append(classes_dict[i])

    return used_classes, weight_dict

used_classes, weight_dict = count_classes_weights(node_labels)

Dataframe_Labels['weight'] = [weight_dict[x] for x in Dataframe_Labels['class_int']]

node_weights = torch.tensor(Dataframe_Labels['weight'], dtype=torch.float)
node_weights


tensor([ 1.4465,  1.4465,  1.4465,  1.2007,  1.2007,  3.4662,  3.4662,  4.0825,
         1.4465,  1.2007,  3.4662,  1.4465,  1.2007,  1.4465,  3.4662,  1.2007,
         1.2007,  1.4465,  1.2007,  1.2007,  3.4662,  1.4465,  3.4662,  9.1855,
         1.2007,  1.2007, 30.6185,  1.2007,  1.2007,  1.2007,  1.2007,  1.2007,
         1.2007,  1.4465,  4.0825,  4.0825,  1.4465,  1.2007,  1.2007,  3.4662,
         1.2007, 30.6185,  9.1855,  1.2007,  1.4465,  1.2007,  1.2007,  1.4465,
         1.2007,  1.2007, 30.6185,  1.2007,  1.2007,  1.4465,  1.2007,  1.4465,
         1.2007,  1.4465,  1.4465,  1.4465,  1.4465,  1.4465,  1.4465,  1.4465,
         1.4465,  1.4465,  1.4465,  1.4465,  1.4465,  1.4465,  1.4465,  1.4465,
         1.4465,  1.4465,  1.4465,  1.4465,  1.4465,  1.2007,  1.4465,  3.4662,
         1.2007,  3.4662,  1.4465,  1.4465,  1.2007,  3.4662,  1.2007,  3.4662,
         1.4465,  1.4465,  1.2007,  3.4662,  9.1855,  1.2007,  4.0825,  1.4465,
         4.0825,  9.1855,  1.2007,  1.20

# Définition des masques utilisés

In [133]:
def set_mask(start, length=0, to_end=False):
    mask = []
    for i in range(404):
        if (i < start or i >= start + length) and not to_end: 
            mask.append(False)
        else : 
            mask.append(True)
    return mask

In [134]:
# data on which the model will be trained
train_length = int(404 * 0.7)
train_mask = torch.tensor(set_mask(start=0, length=train_length), dtype=torch.bool)

length = int((404 - train_length) * 0.5)
val_mask = torch.tensor(set_mask(start=train_length, length=length), dtype=torch.bool)

test_mask = torch.tensor(set_mask(start=train_length + length, length=length), dtype=torch.bool)

train_mask

tensor([ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, 

# Définition de l'objet data utilisé

In [135]:
data1 = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=node_labels, weights = node_weights, train_mask=train_mask, val_mask=val_mask, test_mask=test_mask, num_classes = 6)
print(data1)

print(len(data1.y.tolist()))
print(data1.y.tolist())
print(data1.train_mask.tolist())
print(data1.y[data1.train_mask].tolist())
print(data1.y.tolist())
print(data1.test_mask.tolist())
print(data1.y[data1.test_mask].tolist())
print(data1.y.tolist())
print(data1.val_mask.tolist())
print(data1.y[data1.val_mask].tolist())

print(data1.x)
print(data1.edge_index)

Data(x=[404, 825], edge_index=[2, 49308], edge_attr=[49308], y=[404], weights=[404], train_mask=[404], val_mask=[404], test_mask=[404], num_classes=6)
404
[0, 0, 0, 1, 1, 2, 2, 3, 0, 1, 2, 0, 1, 0, 2, 1, 1, 0, 1, 1, 2, 0, 2, 4, 1, 1, 5, 1, 1, 1, 1, 1, 1, 0, 3, 3, 0, 1, 1, 2, 1, 5, 4, 1, 0, 1, 1, 0, 1, 1, 5, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 2, 0, 0, 1, 2, 1, 2, 0, 0, 1, 2, 4, 1, 3, 0, 3, 4, 1, 1, 1, 4, 2, 1, 1, 3, 0, 4, 1, 2, 1, 1, 3, 0, 1, 0, 0, 2, 4, 1, 0, 1, 1, 0, 1, 1, 1, 3, 2, 5, 0, 0, 1, 2, 0, 0, 1, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 1, 0, 1, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 3, 3, 1, 1, 1, 3, 2, 1, 2, 2, 3, 3, 0, 1, 0, 3, 1, 1, 0, 3, 1, 3, 3, 1, 1, 0, 1, 1, 4, 3, 3, 1, 0, 3, 3, 1, 4, 1, 2, 0, 2, 3, 0, 1, 0, 1, 1, 0, 5, 0, 1, 1, 0, 2, 0, 1, 0, 2, 0, 1, 3, 0, 1, 0, 1, 1, 2, 1, 2, 4, 4, 1, 1, 0, 2, 2, 1, 0, 1, 0, 1, 1, 0, 3, 2, 1, 4, 0, 2, 2, 0, 3, 2, 0, 0, 1, 2, 0, 2, 0, 0, 2, 1, 1, 3, 4, 1, 1, 1, 0, 1, 1

In [136]:
data = data1

# 1er GATv2 - Simple

In [137]:
class GATv2(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super(GATv2, self).__init__()
        torch.manual_seed(1234)
        self.conv1 = GATv2Conv(data.num_features, hidden_channels, heads=heads, edge_dim=1)
        self.conv2 = GATv2Conv(hidden_channels * heads, 6, edge_dim=1)

    def forward(self, x, edge_index, edge_attr):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index, edge_attr)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return x
    

model = GATv2(hidden_channels=16, heads=8)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test(mask):
    print("test", len(mask))
    model.eval()
    out = model(data.x, data.edge_index, data.edge_attr)
    #print(f"out shape: {out.shape}")
    pred = out.argmax(dim=1)
    #print(pred[mask])
    correct = pred[mask] == data.y[mask]
    acc = int(correct.sum()) / int(mask.sum())
    return acc , pred

for epoch in range(1, 2):
    loss = train()
    val_acc = test(data.val_mask)[0]
    test_acc, pred = test(data.test_mask)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val {val_acc:.4f}, Test {test_acc:.4f}')

GATv2(
  (conv1): GATv2Conv(825, 16, heads=8)
  (conv2): GATv2Conv(128, 6, heads=1)
)
test 404
test 404
Epoch: 001, Loss: 2.0898, Val 0.2623, Test 0.2951


In [138]:
print(data.y[data.test_mask])
print(pred[test_mask])

tensor([1, 1, 1, 1, 1, 1, 2, 1, 3, 2, 0, 3, 1, 1, 2, 4, 0, 1, 0, 1, 1, 1, 1, 5,
        3, 0, 0, 1, 0, 1, 4, 0, 0, 0, 0, 0, 0, 3, 3, 1, 1, 1, 0, 0, 1, 1, 1, 2,
        0, 1, 1, 0, 0, 3, 1, 1, 1, 1, 1, 0, 3])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


# 2eme GATv2 - Avec mise à jour des poids des classes

In [139]:
import random
test_index = random.sample(range(0, data.num_nodes), int(0.3 * data.num_nodes))
print(test_index)
data.test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.test_mask[test_index] = True

[113, 96, 50, 192, 330, 182, 366, 252, 225, 278, 128, 202, 167, 142, 392, 247, 365, 232, 139, 342, 230, 244, 358, 197, 394, 215, 54, 279, 119, 369, 82, 203, 250, 98, 90, 211, 338, 402, 327, 186, 33, 0, 209, 189, 273, 103, 40, 66, 259, 205, 176, 23, 72, 263, 381, 69, 11, 376, 315, 227, 360, 41, 116, 73, 65, 42, 100, 166, 28, 325, 286, 86, 51, 184, 67, 351, 93, 380, 154, 111, 110, 332, 29, 260, 157, 359, 257, 212, 60, 379, 334, 196, 298, 213, 294, 155, 364, 49, 305, 173, 357, 285, 7, 174, 345, 214, 146, 59, 333, 386, 208, 48, 13, 217, 130, 304, 387, 105, 316, 195, 297]


In [140]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
import seaborn as sn
import random

class GATv2(torch.nn.Module):
    def __init__(self, hidden_channels, heads):
        super(GATv2, self).__init__()
        torch.manual_seed(1234)
        self.conv1 = GATv2Conv(data.num_features, hidden_channels, heads=heads, edge_dim=1)
        self.conv2 = GATv2Conv(hidden_channels * heads, data.num_classes, edge_dim=1)
        
    def forward(self, x, edge_index, edge_attr):
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index, edge_attr)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index, edge_attr)
        return x

def weighted_cross_entropy_loss(output, target, weights):
    loss = F.cross_entropy(output, target, reduction='none')
    weighted_loss = loss * weights[target]
    return weighted_loss.mean()

def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_attr)
    loss = weighted_cross_entropy_loss(out[data.train_mask], data.y[data.train_mask], data.weights[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss, model


def test(model, data, mask):
    model.eval()
    out = model(data.x, data.edge_index, data.edge_attr)
    pred = out.argmax(dim=1)
    correct = pred[mask] == data.y[mask]
    acc = int(correct.sum()) / int(mask.sum())
    return acc, pred[mask]

def complete(dict):
    classes = ['LumP', 'Ba/Sq', 'LumU', 'Stroma-rich', 'LumNS', 'NE-like']
    for i in classes:
        if not dict[i]:
            dict[i] = 0

def dinstinction(data):
    train_val_indexes = torch.ones(data.num_nodes, dtype=torch.bool)
    for i in range(len(train_val_indexes)):
        if data.test_mask[i]:
            train_val_indexes[i] = False

    X=data.x[train_val_indexes]
    Y=data.y[train_val_indexes]
    return(X,Y)

def cross_validation(data, k_folds, hidden_channels, heads, num_epochs=1000):
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=1234)
    all_val_acc = []

    val_acc_best = 0

    data_X_for_Val, data_Y_for_val  = dinstinction(data)

    for fold, (train_index, val_index) in enumerate(skf.split(data_X_for_Val, data_Y_for_val)):
        # Define masks
        data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        data.train_mask[train_index] = True
        
        data.val_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
        data.val_mask[val_index] = True
        
        train_classes, _ = count_classes_weights(data.y[data.train_mask])
        val_classes, _ = count_classes_weights(data.y[data.val_mask])

        print(train_classes, len(train_classes))
        print(val_classes, len(val_classes))

        # Initialize model, optimizer, and loss function
        model = GATv2(hidden_channels=hidden_channels, heads=heads)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

        lost_function_compil = []
        all_val_acc_plot = []

        # Training loop
        for epoch in range(1, num_epochs):
            loss, model = train(model, data, optimizer)
            train_acc, _ = test (model, data, data.train_mask)
            val_acc, _ = test(model, data, data.val_mask)
            lost_function_compil.append(loss.detach().tolist())
            all_val_acc_plot.append(val_acc)

            print(f'Fold: {fold + 1}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Train_acc {train_acc:.4f}, Val_acc {val_acc:.4f}')

        # Evaluate on validation set
        val_acc, y_pred = test(model, data, data.val_mask)
        y_true = data.y[data.val_mask]

        # Store results
        all_val_acc.append(val_acc)
        conf_matrix = confusion_matrix(y_true.cpu().tolist(), y_pred.cpu().tolist())

        if val_acc > val_acc_best:
            best_model = model

        # Plot confusion matrix

        labels=['LumP', 'Ba/Sq', 'LumU', 'Stroma-rich', 'LumNS', 'NE-like']

        conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
        df_cm = pd.DataFrame(conf_matrix_normalized, index=labels, columns=labels)  
        sn.heatmap(df_cm, annot=True)
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.title('Confusion Matrix on Validation')
        plt.show()

        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

        ax1.plot(range(len(lost_function_compil)), lost_function_compil, label='Loss')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Loss')
        ax1.set_title('Loss Function over Epochs')
        ax1.legend()

        ax2.plot(range(len(all_val_acc_plot)), all_val_acc_plot, label='val_accuracy')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('val_accuracy')
        ax2.set_title('val_accuracy over Epochs')
        ax2.legend()

        plt.show()

    # Calculate and print overall metrics
    mean_val_acc = np.mean(all_val_acc)
    std_val_acc = np.std(all_val_acc)
    print(f'Mean Validation Accuracy: {mean_val_acc:.4f}, Std Validation Accuracy: {std_val_acc:.4f}')

    return best_model
    
best_model = cross_validation(data, k_folds=10, hidden_channels=20, heads=8, num_epochs=1)

"""labels=['LumP', 'Ba/Sq', 'LumU', 'Stroma-rich', 'LumNS', 'NE-like']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.pie(train_classes, labels=labels, startangle=90, autopct='%1.1f%%')
ax1.axis('equal')
ax1.set_title('Cancer repartition for training', fontsize=14)

ax2.pie(val_classes, labels=labels, startangle=90, autopct='%1.1f%%')
ax2.axis('equal')
ax2.set_title('Cancer repartition for validation', fontsize=14)"""

['LumP', 'Ba/Sq', 'LumU', 'Stroma-rich', 'LumNS', 'NE-like'] 6
['LumP', 'Ba/Sq', 'LumU', 'Stroma-rich'] 4


/home/remik/Documents/Cassiopée/cassiopee-projet/Cass/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(
/tmp/ipykernel_39388/3443011196.py:114: RuntimeWarning: invalid value encountered in divide
  conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]


ValueError: Shape of passed values is (5, 5), indices imply (6, 6)

In [ ]:
print(data.train_mask)
print(data.val_mask)
print(data.test_mask)

tensor([False,  True,  True,  True,  True,  True,  True,  True, False,  True,
         True,  True,  True,  True,  True,  True, False,  True,  True,  True,
         True, False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True, False, False,  True,  True,  True,  True,  True, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True, False,  True,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
        False, False,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True, 

In [ ]:
test_acc, y_pred = test(best_model, data, data.test_mask)
y_true = data.y[data.test_mask]

conf_matrix = confusion_matrix(y_true.cpu().tolist(), y_pred.cpu().tolist())

print(f'Test accuracy: {test_acc:.4f}')

classes = ('LumP', 'Ba/Sq', 'LumU', 'Stroma-rich', 'LumNS', 'NE-like')
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
df_cm = pd.DataFrame(conf_matrix_normalized, index=classes, columns=classes)  
sn.heatmap(df_cm, annot=True)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix on Test')
plt.show()

Test accuracy: 0.2893


ValueError: Shape of passed values is (5, 5), indices imply (6, 6)